In [23]:
import numpy as np

In [50]:
Horizon = 5
P = np.array([[0.9, 0.1], [0.1, 0.9]])
cost = 0.3

In [51]:
numStates = 2*Horizon*Horizon
numActions = 2  #(0 refers No Probe and 1 refers to Probe)
gamma = 1
def tuple_to_state (a,b,c):
    #print(a*2*Horizon + b*Horizon + c)
    return (a*2*(Horizon) + b*Horizon + c)
def state_to_tuple(v):
    return()
# the state tuples are arranged st 
#  state count           tuple 
#        0               (0,0,0)
#        1               (0,0,1)
#        H-1             (0,0,H-1)
#        H               (0,1,0)
#        H+1             (0,1,1)
#        2H-1            (0,1,H-1)
#        2H              (1,0,0)
#        2H+1            (1,0,1)
#        2H^2-1          (1,1,H-1)    

In [52]:
P_exp = np.zeros((Horizon, P.shape[0], P.shape[1]))
for i in range(Horizon):
    if i == 0:
        P_exp[0] = P
    else : 
        P_exp[i] = P_exp[i-1]@P
# print(P_exp[:,:,2])

In [53]:
transitions = np.zeros((numStates,numActions,numStates))
rewards = np.ones((numStates,numActions,numStates)) * -1000

In [62]:
for lpr in range(2):
    transitions[tuple_to_state(0,lpr,0)][0][tuple_to_state(1,lpr,0)] = 1
    rewards[tuple_to_state(0,lpr,0)][0][tuple_to_state(1,lpr,0)] = -np.min(P_exp[0][lpr])
for t in range(Horizon-1):
    for lpr in range(2):
        for lpt in range(t):
            transitions[tuple_to_state(t,lpr,lpt)][0][tuple_to_state(t+1,lpr,lpt)] = 1
            transitions[tuple_to_state(t,lpr,lpt)][1][tuple_to_state(t+1,0,t)] = P_exp[t-lpt][lpr][0]
            transitions[tuple_to_state(t,lpr,lpt)][1][tuple_to_state(t+1,1,t)] = P_exp[t-lpt][lpr][1]
            rewards[tuple_to_state(t,lpr,lpt)][1][tuple_to_state(t+1,0,t)] = -cost
            rewards[tuple_to_state(t,lpr,lpt)][1][tuple_to_state(t+1,1,t)] = -cost
            rewards[tuple_to_state(t,lpr,lpt)][0][tuple_to_state(t+1,lpr,lpt)] = -np.min(P_exp[t-lpt][lpr])


In [45]:
print(np.max(transitions))
print(np.max(rewards))
print((rewards + gamma*np.zeros(numStates)).shape)
print((transitions * (rewards + gamma*np.zeros(numStates))).shape)
print(rewards)

1.0
-0.18000000000000002
(5000, 2, 5000)
(5000, 2, 5000)
[[[-1000. -1000. -1000. ... -1000. -1000. -1000.]
  [-1000. -1000. -1000. ... -1000. -1000. -1000.]]

 [[-1000. -1000. -1000. ... -1000. -1000. -1000.]
  [-1000. -1000. -1000. ... -1000. -1000. -1000.]]

 [[-1000. -1000. -1000. ... -1000. -1000. -1000.]
  [-1000. -1000. -1000. ... -1000. -1000. -1000.]]

 ...

 [[-1000. -1000. -1000. ... -1000. -1000. -1000.]
  [-1000. -1000. -1000. ... -1000. -1000. -1000.]]

 [[-1000. -1000. -1000. ... -1000. -1000. -1000.]
  [-1000. -1000. -1000. ... -1000. -1000. -1000.]]

 [[-1000. -1000. -1000. ... -1000. -1000. -1000.]
  [-1000. -1000. -1000. ... -1000. -1000. -1000.]]]


In [45]:
def vi_solve():
    V = np.zeros(numStates)
    V_old = V
    temp = np.zeros((numStates,numActions))
    while (1):
        temp = np.sum(transitions * (rewards + gamma*V_old), axis=2)
        V = np.max(temp, axis=1)
        if np.allclose(V, V_old,0,1e-10):
            break
        V_old = V
    policy = np.argmax(temp, axis=1)
    return V, policy

In [46]:
V, policy = vi_solve()
print(V, policy)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.] [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 1 0 0 0 1 1 0 0 0 1 1 1 0 0 1 1
 1 0 0 0 0 0 0 0 0 0 0 0 0]


In [48]:
print(np.sum(policy))

12


In [60]:
def find_value(mdp, Policy_fn):
    T_pi = mdp.transitions[np.arange(mdp.numStates), Policy_fn]
    R_pi = mdp.rewards[np.arange(mdp.numStates), np.array(Policy_fn)]
    V = np.squeeze(np.matmul(np.linalg.inv(np.eye(mdp.numStates) - mdp.gamma * T_pi),np.sum(T_pi * R_pi, axis=1)))
    #print("debug[0]:",V)
    return V

def hpi_solve(mdp):
    pi = np.random.randint(low=0, high=mdp.numActions, size=mdp.numStates)
    pi_old = pi
    while (1):
        V = find_value(mdp, pi_old)
        pi = np.argmax(np.sum(mdp.transitions * (mdp.rewards + mdp.gamma*V), axis=2), axis=1)
        if np.array_equal(pi, pi_old):
            break
        pi_old = pi
    return V, pi

In [57]:
def loss_cal(prob,i,j,state): #i is position of last query, j is last state at which we need loss
    
    n=prob.shape[1]
    loss = np.zeros((n,1));
    # print(loss.shape)
    for x in range(j-i):
        temp = np.reshape(np.min(np.linalg.matrix_power(prob,x+1),axis=1),(2,1))
        # print(temp.shape)
        loss+= temp
    return np.matmul(state,loss)
def OPTPolicy(prob,T,cost):
    n = prob.shape[0]
    
    dp = np.zeros((T+1,n)) # dp_ik is the position to query next if previous query was at i and result was state k   
    loss = np.zeros((T+1,n)) # Expected loss till the end of the horizon
    for i in range(n):
        dp[T][i]=np.inf
        loss[T][i]=0

    for i in range(T-1,-1,-1):
        for k in range(n):
            state = np.zeros((1,n))
            state[0,k]=1
            curr=loss_cal(prob,i,T,state)
            dp[i,k]=np.inf
            for x in range(i+1,T+1):
                new_curr = loss_cal(prob,i,x-1,state) + cost
                
                state = np.matmul(state,np.linalg.matrix_power(prob,x-i))
                for s in range(n):
                    new_curr += loss[x,s]*state[0,s]

                if new_curr < curr:
                    curr=new_curr
                    dp[i,k]=x
                    
            loss[i,k]=curr
    return np.min(loss[0,0])  
    print("Expected Loss: ",loss[0,0])

In [65]:
class MDP:
    def __init__(self,numActions,numStates,rewards,transitions,gamma):
        self.numActions=numActions
        self.numStates=numStates
        self.rewards=rewards
        self.transitions=transitions
        self.gamma=gamma
mdp=MDP(numActions,numStates,rewards,transitions,1)
V, policy = hpi_solve(mdp)
print("hpi done")
print(-1*V[tuple_to_state(1,0,0)]+np.min(P_exp[0][0]),OPTPolicy(P,Horizon,cost))

hpi done
0.7600000000000001 0.8600000000000002
